In [2]:
import torch
import numpy as np
import random

# https://docs.pytorch.org/docs/stable/notes/randomness.html
RAND_SEED = 10241
torch.manual_seed(RAND_SEED)
random.seed(RAND_SEED)
np.random.seed(RAND_SEED)

# linear module impl

**To be able to reproduce the same result across different executions, ensure we use specified seeds for places which entails use of randomness (e.g. initialization of model weights)**.

In [1]:
import torch
import torch.nn as nn
from einops import einsum

class Linear(nn.Module):
    def __init__(self, in_features: int, out_features: int, device: torch.device | None = None, dtype: torch.dtype =None) -> None:
        super().__init__()
        std = (2 / (in_features + out_features)) ** 0.5
        self.w = nn.Parameter(nn.init.trunc_normal_(torch.empty(out_features, in_features, device=device, dtype=dtype), mean=0, std=std, a=-3*std, b=3*std))
        print(self.w.device)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        #return einsum(x, self.w, '... d_in , d_out d_in -> ... d_out')
        return x @ self.w.T

In [77]:
lxf = Linear(5, 2, device='cpu', dtype=torch.float32)

x = torch.randn(2, 5, device='cpu')
lxf.forward(x)

cpu


tensor([[ 0.1795, -0.2127],
        [ 1.8593, -2.1613]], grad_fn=<MmBackward0>)

In [70]:
help(torch.empty)

Help on built-in function empty in module torch:

empty(...)
    empty(*size, *, out=None, dtype=None, layout=torch.strided, device=None, requires_grad=False, pin_memory=False, memory_format=torch.contiguous_format) -> Tensor

    Returns a tensor filled with uninitialized data. The shape of the tensor is
    defined by the variable argument :attr:`size`.

    .. note::
        If :func:`torch.use_deterministic_algorithms()` and
        :attr:`torch.utils.deterministic.fill_uninitialized_memory` are both set to
        ``True``, the output tensor is initialized to prevent any possible
        nondeterministic behavior from using the data as an input to an operation.
        Floating point and complex tensors are filled with NaN, and integer tensors
        are filled with the maximum value.

    Args:
        size (int...): a sequence of integers defining the shape of the output tensor.
            Can be a variable number of arguments or a collection like a list or tuple.

    Keyword

# Embedding layer impl

In [34]:
import torch
import torch.nn as nn
from einops import reduce

class Embedding(nn.Module):
    def __init__(
            self,
        num_embeddings: int,
        embedding_dim: int,
        device: torch.device | None = None,
        dtype: torch.dtype | None = None,
) -> None:
        '''
        num_embeddings: Vocabulary size
        embedding_dim: Dimension of the embedding vectors
        device: Device to store model parameters on
        dtype: Model parameter datatype

        See https://docs.pytorch.org/docs/stable/generated/torch.nn.Embedding.html
        '''
        super().__init__()
        self.embedding = nn.Parameter(nn.init.trunc_normal_(
            torch.empty(num_embeddings, embedding_dim, dtype=dtype), mean=0, std=1, a=-3, b=3))
        
    def forward(self, token_ids: torch.Tensor) -> torch.Tensor:
        '''
        token_ids: A tensor of long type of shape (batch, seq_len)
        
        Spec:
        Embedding layer serves as a mapping from individual token (identified
        by its id) to corresponding embedding vector (of size `embedding_dim`)

        So looking up embedding vector by given token id, assemble the found
        vectors into a matrix then return is what embedding layer needs to do.

        The resulting matrix will be of shape (batch, seq_len, embedding_dim)

        To construct this matrix in imperative, non-linear algebra manner:
            For each seq in batch:
                For each token in seq:
                    Get vector at self.embedding[token's ID] and add it to
                        resulting matrix

        The goal is to do this in pytorch-idiomatic, linear algebra way.

        Use advanced indexing:
        - https://numpy.org/doc/stable/user/basics.indexing.html
        - https://numpy.org/neps/nep-0021-advanced-indexing.html
        - https://blog.ezyang.com/2019/05/pytorch-internals/
        '''
        return self.embedding[token_ids]


In [45]:
token_seq_batch = torch.randint(low=0, high=10, size=(2, 3), dtype=torch.long)

In [46]:
token_seq_batch

tensor([[2, 9, 2],
        [2, 1, 6]])

In [35]:
vocab_size, embedding_dim = 10, 3
toy_embeddings = nn.init.trunc_normal_(torch.empty(vocab_size, embedding_dim), mean=0, std=1, a=-3, b=3)
toy_embeddings

tensor([[-0.6658,  0.0909,  0.4032],
        [-1.3488, -0.3029,  0.6607],
        [-0.5142, -0.0462, -0.1464],
        [-0.3757,  0.7217, -0.0690],
        [-1.1029, -0.4012,  0.5654],
        [ 0.9065,  1.3091, -1.2292],
        [-1.2851, -0.8448, -0.1809],
        [-1.6742,  0.4734,  0.3509],
        [-1.4873,  1.9627, -0.0351],
        [ 0.3765,  0.0167, -1.7541]])

In [50]:
#token_to_embeddings = torch.empty(2, 3, 3)
#print(f'initial: {token_to_embeddings}')

token_to_embeddings = toy_embeddings[token_seq_batch]

print(f'after: {token_to_embeddings}')

after: tensor([[[-0.5142, -0.0462, -0.1464],
         [ 0.3765,  0.0167, -1.7541],
         [-0.5142, -0.0462, -0.1464]],

        [[-0.5142, -0.0462, -0.1464],
         [-1.3488, -0.3029,  0.6607],
         [-1.2851, -0.8448, -0.1809]]])


In [54]:
token_to_embeddings.ndim

3

# Pre-norm transformer block

## Root Mean Square Layer Normalization

In [36]:
import torch
import torch.nn as nn
from einops import reduce, rearrange

class RMSNorm(nn.Module):
    '''Root Mean Square normalization block.

    
    '''
    def __init__(
        self,
        d_model: int,
        device: torch.device | None = None,
        dtype: torch.dtype | None = None,
) -> None:
        super().__init__()
        self.gains = nn.Parameter(torch.ones(d_model, dtype=dtype, device=device))
        # HYPERPARAM
        self.e = 1e-5
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        '''
        x in shape (batch_size, sequence_length, d_model)
        '''
        in_dtype = x.dtype
        # Q: shall we fix the dtype of model params (gains) to float32?
        x = x.to(torch.float32)
        d_model_size = x.shape[-1]
        # Reduce along x's final dimension of size d_model in square sum
        rms = torch.sqrt(reduce(torch.square(x) , 'b s d_model -> b s', reduction='sum') / d_model_size + self.e)
        # For element-wise division to work we need to satisfy the broadcasting semantics
        # Here rms is in shape (b, s) while the dividend (number to be divided) in shape (b, s, d_model)
        # Thus adjust rms to shape (b, s, 1) for broadcasting to work
        rms = rearrange(rms, 'b s -> b s 1')
        #print(f'rms in shape {rms.shape}: {rms}')
        #print(f'prod in shape {prod.shape}: {prod}')
        res = x * self.gains / rms
        return res.to(in_dtype)

In [37]:
device='mps'
d_model = 2
m = RMSNorm(d_model, device=device)

m.forward(torch.tensor([[[-0.5, 0.3], [0.1, 0.2], [0.1, 0.7]], [[0.4, 0.1], [0.2, 0.1], [0.7, 0.1]]], device=device))

tensor([[[-1.2126,  0.7276],
         [ 0.6323,  1.2647],
         [ 0.2000,  1.4000]],

        [[ 1.3719,  0.3430],
         [ 1.2647,  0.6323],
         [ 1.4000,  0.2000]]], device='mps:0', grad_fn=<DivBackward0>)

In [31]:
t = torch.ones(2, 3)
print(t.element_size())
help(t.element_size)

4
Help on built-in function element_size:

element_size(...) method of torch.Tensor instance
    element_size() -> int

    Returns the size in bytes of an individual element.

    Example::

        >>> torch.tensor([]).element_size()
        4
        >>> torch.tensor([], dtype=torch.uint8).element_size()
        1



In [32]:
torch.finfo(torch.float32)

finfo(resolution=1e-06, min=-3.40282e+38, max=3.40282e+38, eps=1.19209e-07, smallest_normal=1.17549e-38, tiny=1.17549e-38, dtype=float32)

## Position-Wise Feed-Forward Network

In [50]:
import torch
import torch.nn as nn
from einops import einsum

class FFN(nn.Module):
    '''Position-Wise Feed-Forward Network

    
    '''
    def __init__(
        self,
        d_model: int,
        device: torch.device | None = None,
        dtype: torch.dtype | None = None,
) -> None:
        
        super().__init__()
        # Per assignment recommendation, get the multiple of 64 closest to 8/3 x d_model
        d_ff = max(round(8 * d_model / 3 / 64), 1) * 64
        print(f'Computed feed forward network dimension size = {d_ff}')
        self.w1 = nn.Parameter(nn.init.trunc_normal_(
            torch.empty(d_ff, d_model, dtype=dtype, device=device), mean=0, std=1, a=-3, b=3))
        self.w3 = nn.Parameter(nn.init.trunc_normal_(
            torch.empty(d_ff, d_model, dtype=dtype, device=device), mean=0, std=1, a=-3, b=3))
        self.w2 = nn.Parameter(nn.init.trunc_normal_(
            torch.empty(d_model, d_ff, dtype=dtype, device=device), mean=0, std=1, a=-3, b=3))
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        '''
        Spec:

        Different from math notation which is column-major, here we apply matrix operation in row-major manner.
        '''
        # x_w1 = x @ self.w1.T
        x_w1 = einsum(x, self.w1, '... d_model, d_ff d_model -> ... d_ff')
        x_w1_sigmoid = x_w1 * torch.sigmoid(x_w1)
        x_w3 = einsum(x, self.w3, '... d_model, d_ff d_model -> ... d_ff')
        return einsum((x_w1_sigmoid * x_w3), self.w2, '... d_ff, d_model d_ff -> ... d_model')
        

In [47]:
t1 = torch.tensor([[1, 2, 3], [4, 5, 6]], dtype=torch.float)
t2 = torch.tensor([0.1,0.2,0.3])
t1 @ t2

tensor([1.4000, 3.2000])

## RoPE - Now it becomes crucial to understand and document the shape of each layer's input and output :D

In [126]:
from math import sin, cos
import torch
from torch import Tensor
import torch.nn as nn
from einops import rearrange, einsum
from jaxtyping import Float


class RotaryPositionalEmbedding(nn.Module):
    def __init__(self, theta: float, d_k: int, max_seq_len: int, device=None) -> None:
        """
        This module is used in transformer model component "Causal Multi-Head Self-Attention w/ RoPE"
        Its input is the output of pre-norm layer, in shape (batch_size, seq_len, d_embedding)

        This is how we model positions in a transformer, aka encoding position-specific
        info into the transformer model.
        See https://youtu.be/ptFiH_bHnJw?list=PLoROMvodv4rOY23Y0BoGoBGgQ1zmU_MT_&t=2007

        This is to construct a mathematical tool to measure how "far" a pair of tokens
        are away from each other, aka *relative* position b/w a pair of token.

        The idea is to use the observation that *inner product of two vector is invariant
        to arbitrary rotation* to represent our desired characteristic that token embedding
        should be invariant to absolute position.

        d_k: dimension of query and key vectors, = d_embedding
        (Note it seems we assume d_k to be a even number)

        max_seq_len: Maximum sequence length. Used only for caching pre-computed rotation
        angle values. Not necessarily = seq_len

        spec:
        i: token position index (in a token sequence). IMO i in [0, max_seq_len)
            Q: Does this index start from 0 or 1? Assignment handout doesn't tell. But
            per my experience in math notation indexing starting from 1 seems prevailing
            A: Per UT, i starts from 0 (UT will barf if we use 1-based index)

        R: Matrix that contain pair-wise rotation matrix R_i, where i in [0, max_seq_len)
            R_i is of shape (d_k, d_k), so R is of shape (max_seq_len, d_k, d_k) to
            account for ALL token positions in a token sequence.

        The constructor logic is to compute and cache R.

        If we did this in sequential manner:
        For i in [0, max_seq_len):
            Compute R_i.
            For k in [1, d_k/2]:
                Compute rotation angle theta_i_k
                    theta_i_k = (i+1) / (theta^((2k-2)/d_k))
                Compute cos and sin value at theta_i_k, form 2x2 matrix block R_i_k
                Stack the block diagonally into R_i
            R[i] = R_i

        TODO: More pytorch-idiomatic, parallelizable impl
        """
        super().__init__()
        assert d_k % 2 == 0, f"{d_k} is not even number"

        token_positions = torch.arange(max_seq_len, device=device, dtype=dtype)
        # k in [1, 2, 3, ..., d/2] -> 2k-2 in [0, 2, 4, ..., d-2]
        inv_theta_exps = theta ** (-torch.arange(start=0, end=d_k, step=2, device=device, dtype=dtype) / d_k)
        # angle matrix of shape (max_seq_len, d_k/2)
        angles = token_positions[:, None] * inv_theta_exps[None, :]
        # sin and cos of angles (max_seq_len, d_k/2)
        sin, cos = torch.sin(angles), torch.cos(angles)
        R = torch.empty(max_seq_len, d_k, d_k, dtype=dtype, device=device)
        indices_on_diag = torch.arange(start=0, end=d_k, step=2, dtype=dtype, device=device)
        R[:, indices_on_diag, indices_on_diag] = cos
        R[:, indices_on_diag+1, indices_on_diag] = sin
        R[:, indices_on_diag, indices_on_diag+1] = -sin
        R[:, indices_on_diag+1, indices_on_diag+1] = cos

        # R
        self.register_buffer(
            "rotation_matrix",
            R,
            persistent=False,
        )

    def forward(self, x: torch.Tensor, token_positions: torch.Tensor) -> torch.Tensor:
        """
        x: tensor of shape (..., seq_len, d_k)
        token_positions: a tensor of shape (..., seq_len) specifying the token
            positions of x along the sequence dimension.
        return: a tensor of the same shape as x.

        Spec:
        Identify the rotation matrices to use by given token position.
            Do this w/ Pytorch's advanced indexing: R[token_positions] -> tensor of shape (..., seq_len, d_k, d_k)
        Multiply a token's embedding vector of shape (d_k,) w/ the corresponding identified rotation matrix of shape (d_k, d_k)
        """
        rotation_matrix_by_pos: Float[Tensor, "... d_k d_k"] = self.rotation_matrix[
            token_positions
        ]
        # note in einsum notation we cannot use duplicated
        return einsum(
            x,
            rotation_matrix_by_pos,
            # NOTE again in implementation we have to use row-major order,
            # aka token_embedding_vector @ R_i.T Otherwise the result will
            # be incorrect.
            "... d_k_in, ... d_k_out d_k_in -> ... d_k_out",
        )

In [116]:
max_seq_len = 4
d_k=6
theta = 2
device, dtype = 'mps', torch.float32
token_positions = torch.arange(max_seq_len, device=device, dtype=dtype)
print(token_positions.shape)
# k in [1, 2, 3, ..., d/2] -> 2k-2 in [0, 2, 4, ..., d-2]
inv_theta_exps = theta ** (-torch.arange(start=0, end=d_k, step=2, device=device, dtype=dtype) / d_k)
print(inv_theta_exps.shape)
# angle matrix of shape (max_seq_len, d_k / 2)
angles = token_positions[:, None] * inv_theta_exps[None, :]
print(torch.equal(angles, token_positions[:, None] @ inv_theta_exps[None, :]))
angles

torch.Size([4])
torch.Size([3])
True


tensor([[0.0000, 0.0000, 0.0000],
        [1.0000, 0.7937, 0.6300],
        [2.0000, 1.5874, 1.2599],
        [3.0000, 2.3811, 1.8899]], device='mps:0')

In [111]:
torch.equal(token_positions.unsqueeze(dim=1), token_positions[:, None])
token_positions[:, None]

tensor([[0.],
        [1.],
        [2.],
        [3.]], device='mps:0')

In [112]:
torch.equal(inv_theta_exps[None, :], inv_theta_exps.unsqueeze(dim=0))
inv_theta_exps[None, :]

tensor([[1.0000, 0.7937, 0.6300]], device='mps:0')

In [120]:
sin, cos = torch.sin(angles), torch.cos(angles)
print(sin)
print(cos)

tensor([[0.0000, 0.0000, 0.0000],
        [0.8415, 0.7130, 0.5891],
        [0.9093, 0.9999, 0.9521],
        [0.1411, 0.6893, 0.9495]], device='mps:0')
tensor([[ 1.0000,  1.0000,  1.0000],
        [ 0.5403,  0.7012,  0.8081],
        [-0.4161, -0.0166,  0.3059],
        [-0.9900, -0.7245, -0.3137]], device='mps:0')


In [124]:
R = torch.zeros(max_seq_len, d_k, d_k, device='mps')
idx = torch.arange(0, d_k, 2)
print(idx)
R[:, idx, idx] = cos

tensor([0, 2, 4])


In [156]:
from einops import rearrange

torch.manual_seed(43)
t1 = torch.reshape(torch.arange(0, 20), (2, 2, 5))
print(t1, t1.shape)
t2 = rearrange(torch.arange(0, 2, step=0.1).reshape(2, 2, 5), 'b s d -> b 1 s d')
print(t2, t2.shape)

tensor([[[ 0,  1,  2,  3,  4],
         [ 5,  6,  7,  8,  9]],

        [[10, 11, 12, 13, 14],
         [15, 16, 17, 18, 19]]]) torch.Size([2, 2, 5])
tensor([[[[0.0000, 0.1000, 0.2000, 0.3000, 0.4000],
          [0.5000, 0.6000, 0.7000, 0.8000, 0.9000]]],


        [[[1.0000, 1.1000, 1.2000, 1.3000, 1.4000],
          [1.5000, 1.6000, 1.7000, 1.8000, 1.9000]]]]) torch.Size([2, 1, 2, 5])


In [158]:
# The element-wise multiplication below uses broadcasting.
# For how it works, see https://numpy.org/doc/stable/user/basics.broadcasting.html
t3 = t1 * t2
t3, t3.shape

(tensor([[[[ 0.0000,  0.1000,  0.4000,  0.9000,  1.6000],
           [ 2.5000,  3.6000,  4.9000,  6.4000,  8.1000]],
 
          [[ 0.0000,  1.1000,  2.4000,  3.9000,  5.6000],
           [ 7.5000,  9.6000, 11.9000, 14.4000, 17.1000]]],
 
 
         [[[ 0.0000,  1.1000,  2.4000,  3.9000,  5.6000],
           [ 7.5000,  9.6000, 11.9000, 14.4000, 17.1000]],
 
          [[10.0000, 12.1000, 14.4000, 16.9000, 19.6000],
           [22.5000, 25.6000, 28.9000, 32.4000, 36.1000]]]]),
 torch.Size([2, 2, 2, 5]))

## Softmax function

In [199]:
import torch


def softmax(x: torch.Tensor, i: int) -> torch.Tensor:
    """
    x: Multi-dimensional tensor.
    i: The i-th dimension of x (For now assume the index is 0-based)
    return: Tensor of same shape as x, w/ values at i-th dimension turn into
        normalized probabilities under softmax.

    Can refer to the behavior of std pytorch lib softmax function.
    """
    assert (
        -x.ndim <= i < x.ndim
    ), f"Given dimension index {i} not found in tensor of {x.ndim} dimensions"
    # Get the max values and corresponding indices on dimension i, plus keeping
    # x's dimension around. This saves the extra move of creating a new
    # dimensin in the resulting tensor (eg via tensor.unsqueeze(dim=i))
    max_on_dim_i = x.max(dim=i, keepdim=True)
    # softmax formula
    # Subtract values on dimension i by the max values so that exp()
    # of these values yields small values in (0, 1] for numerical stability
    exp = torch.exp(x - max_on_dim_i.values)
    exp_sum_on_dim_i = torch.sum(exp, dim=i, keepdim=True)
    return exp / exp_sum_on_dim_i

In [88]:
re = sm(torch.randint(0, 24, (3, 4, 2)).to(torch.float32), i=1)
print(re)
print(torch.sum(re[:, :, ::], dim=1))

tensor([[[4.5397e-05, 1.7986e-02],
         [3.0588e-07, 6.0337e-06],
         [1.6701e-05, 9.8201e-01],
         [9.9994e-01, 2.0241e-09]],

        [[6.6767e-10, 9.5256e-01],
         [2.9539e-04, 1.5909e-05],
         [8.8054e-01, 4.7425e-02],
         [1.1917e-01, 7.2228e-10]],

        [[1.2202e-05, 1.2328e-04],
         [6.6619e-04, 9.1090e-04],
         [2.6876e-01, 9.9892e-01],
         [7.3056e-01, 4.5351e-05]]])
tensor([[1.0000, 1.0000],
        [1.0000, 1.0000],
        [1.0000, 1.0000]])


### Observing the behavior of software function in std Pytorch lib provides clue of impl

In [47]:
t = torch.randint(0, 24, (3, 4, 2)).to(torch.float32)
print(t.shape)
t

torch.Size([3, 4, 2])


tensor([[[14.,  6.],
         [ 9., 13.],
         [ 5., 17.],
         [21., 20.]],

        [[ 6., 19.],
         [12., 11.],
         [ 4., 12.],
         [ 9.,  1.]],

        [[10., 22.],
         [13.,  3.],
         [ 5.,  7.],
         [22.,  3.]]])

In [69]:
max_d_2 = t.max(dim=1)
t - max_d_2.values.unsqueeze(dim=1)

tensor([[[ -7., -14.],
         [-12.,  -7.],
         [-16.,  -3.],
         [  0.,   0.]],

        [[ -6.,   0.],
         [  0.,  -8.],
         [ -8.,  -7.],
         [ -3., -18.]],

        [[-12.,   0.],
         [ -9., -19.],
         [-17., -15.],
         [  0., -19.]]])

In [72]:
t / torch.sum(t, dim=2).unsqueeze(dim=2)

tensor([[[0.7000, 0.3000],
         [0.4091, 0.5909],
         [0.2273, 0.7727],
         [0.5122, 0.4878]],

        [[0.2400, 0.7600],
         [0.5217, 0.4783],
         [0.2500, 0.7500],
         [0.9000, 0.1000]],

        [[0.3125, 0.6875],
         [0.8125, 0.1875],
         [0.4167, 0.5833],
         [0.8800, 0.1200]]])

In [35]:
t_sm = torch.softmax(t, dim=2)
print(t_sm)
softmaxed_unit = t_sm[0][0]
print(softmaxed_unit)
print(torch.sum(softmaxed_unit))

tensor([[[9.9995e-01, 4.5398e-05],
         [7.3106e-01, 2.6894e-01],
         [1.2339e-04, 9.9988e-01],
         [5.0000e-01, 5.0000e-01]],

        [[1.7986e-02, 9.8201e-01],
         [1.0000e+00, 2.2603e-06],
         [6.6929e-03, 9.9331e-01],
         [5.0000e-01, 5.0000e-01]],

        [[6.1442e-06, 9.9999e-01],
         [4.7426e-02, 9.5257e-01],
         [9.9988e-01, 1.2339e-04],
         [4.5398e-05, 9.9995e-01]]])
tensor([9.9995e-01, 4.5398e-05])
tensor(1.)


In [36]:
t_sm = torch.softmax(t, dim=1)
print(t_sm)
softmaxed_unit = t_sm[0][:, 0]
print(softmaxed_unit)
print(torch.sum(softmaxed_unit))

tensor([[[1.1850e-01, 5.4000e-06],
         [5.8997e-03, 2.1785e-03],
         [1.4624e-05, 1.1894e-01],
         [8.7559e-01, 8.7887e-01]],

        [[1.1243e-07, 6.1441e-06],
         [9.1105e-04, 2.0611e-09],
         [1.1243e-07, 1.6701e-05],
         [9.9909e-01, 9.9998e-01]],

        [[1.5225e-08, 3.3534e-04],
         [1.1250e-07, 3.0579e-07],
         [9.9966e-01, 1.6696e-05],
         [3.3535e-04, 9.9965e-01]]])
tensor([1.1850e-01, 5.8997e-03, 1.4624e-05, 8.7559e-01])
tensor(1.0000)


In [41]:
t_sm = torch.softmax(t, dim=0)
print(t_sm)
softmaxed_unit = t_sm[:, 0, 0]
print(softmaxed_unit)
print(torch.sum(softmaxed_unit))

softmaxed_unit = t_sm[:, 2, 1]
print(softmaxed_unit)
print(torch.sum(softmaxed_unit))

tensor([[[9.9995e-01, 3.2932e-04],
         [1.1920e-01, 9.9326e-01],
         [2.2603e-06, 9.8670e-01],
         [1.7985e-02, 9.0747e-03]],

        [[4.5398e-05, 1.7980e-02],
         [8.8078e-01, 4.5094e-05],
         [8.3153e-07, 6.6484e-03],
         [9.8197e-01, 4.9546e-01]],

        [[8.3149e-07, 9.8169e-01],
         [1.4711e-05, 6.6925e-03],
         [1.0000e+00, 6.6484e-03],
         [4.4581e-05, 4.9546e-01]]])
tensor([9.9995e-01, 4.5398e-05, 8.3149e-07])
tensor(1.)
tensor([0.9867, 0.0066, 0.0066])
tensor(1.)


## Scaled Dot-Product Attention

In [211]:
from jaxtyping import Float, Bool
import torch
from torch import Tensor, nn

def scaled_dot_product_attention(
    Q: Float[Tensor, '... queries d_k'],
    K: Float[Tensor, '... keys d_k'],
    V: Float[Tensor, '... values d_v'],
    mask: Bool[Tensor, '... queries keys'] | None = None,
) -> Float[Tensor, '... queries d_v']:
    '''
    Your implementation should handle keys and queries of shape
    (batch_size, ..., seq_len, d_k) and values of shape
    (batch_size, ..., seq_len, d_v), where ... represents any
    number of other batch-like dimensions (if provided).
    The implementation should return an output with the
    shape (batch_size, ..., d_v).

    Spec:
    Compute K @ Q.T / sqrt(d_k), yielding a tensor of shape (... keys queries)
    Apply mask to result above if given.
    Apply softmax.
    Matmul result above w/ V, aka V @ (result above).T
    '''
    d_k = K.shape[-1]
    # (... queries keys)
    presoftmax = Q @ K.T / (d_k ** 0.5)
    if mask is not None:
        # mask tensor serves as an index; For positions in index where mask value is false, we assign -inf so that the resulting exp value becomes 0.
        presoftmax[~mask] = -float('inf')

    softmaxed = softmax(presoftmax, dim=-1)
    return softmaxed @ V
    

In [196]:
from einops import repeat

torch.manual_seed(36)
t = torch.randint(0, 6, (2, 3)).to(torch.float32)
mask = torch.tensor([[True, False, False], [False, True, True]])
infs = repeat(torch.tensor([-float('inf')]), 'd -> a (b d)', a=2, b=3)
t, mask, infs

(tensor([[5., 4., 0.],
         [0., 5., 2.]]),
 tensor([[ True, False, False],
         [False,  True,  True]]),
 tensor([[-inf, -inf, -inf],
         [-inf, -inf, -inf]]))

In [191]:
t[mask]

tensor([5, 5, 2])

In [178]:
t * mask

tensor([[0, 0, 0],
        [0, 0, 5]])

In [198]:
t[~mask] = -float('inf')
t

tensor([[5., -inf, -inf],
        [-inf, 5., 2.]])

In [181]:
torch.exp(torch.tensor([-float('inf')]))

tensor([0.])

In [21]:
import torch

torch.manual_seed(41)
t1 = torch.arange(0, 2.4, 0.1).reshape(2, 4, 3).to(torch.float32)
t2 = torch.arange(0, 30, 1).reshape(2, 5, 3).to(torch.float32)
t1 @ t2.transpose(-1, -2)

tensor([[[  0.5000,   1.4000,   2.3000,   3.2000,   4.1000],
         [  1.4000,   5.0000,   8.6000,  12.2000,  15.8000],
         [  2.3000,   8.6000,  14.9000,  21.2000,  27.5000],
         [  3.2000,  12.2000,  21.2000,  30.2000,  39.2000]],

        [[ 62.6000,  74.3000,  86.0000,  97.7000, 109.4000],
         [ 77.0000,  91.4000, 105.8000, 120.2000, 134.6000],
         [ 91.4000, 108.5000, 125.6000, 142.7000, 159.8000],
         [105.8000, 125.6000, 145.4000, 165.2000, 185.0000]]])

In [22]:
re = t1 @ t2.transpose(-1, -2)
re, re.shape

(tensor([[[  0.5000,   1.4000,   2.3000,   3.2000,   4.1000],
          [  1.4000,   5.0000,   8.6000,  12.2000,  15.8000],
          [  2.3000,   8.6000,  14.9000,  21.2000,  27.5000],
          [  3.2000,  12.2000,  21.2000,  30.2000,  39.2000]],
 
         [[ 62.6000,  74.3000,  86.0000,  97.7000, 109.4000],
          [ 77.0000,  91.4000, 105.8000, 120.2000, 134.6000],
          [ 91.4000, 108.5000, 125.6000, 142.7000, 159.8000],
          [105.8000, 125.6000, 145.4000, 165.2000, 185.0000]]]),
 torch.Size([2, 4, 5]))

In [23]:
re[..., 0, :].shape

torch.Size([2, 5])

In [27]:
help(re.element_size)

Help on built-in function element_size:

element_size(...) method of torch.Tensor instance
    element_size() -> int

    Returns the size in bytes of an individual element.

    Example::

        >>> torch.tensor([]).element_size()
        4
        >>> torch.tensor([], dtype=torch.uint8).element_size()
        1



In [31]:
torch.int64.itemsize

8

In [210]:
from einops import einsum

t1, t2 = torch.tensor([0, 2, 4]), torch.tensor([1, -3, 5, -2])
# = column vector of shape (3,) @ row vector (1, 4) = matrix of shape (3, 4)
einsum(t1, t2, 'i, j -> i j')

tensor([[  0,   0,   0,   0],
        [  2,  -6,  10,  -4],
        [  4, -12,  20,  -8]])

## Multi-head self-attention

In [ ]:
from jaxtyping import Float, Bool
import torch
from torch import Tensor, nn


class MultiheadSelfAttention(nn.Module):
    def __init__(
        self,
        d_model: int,
        num_heads: int,
        theta: float | None = None,
        max_seq_len = int | None = None,
        device: torch.device | None = None,
        dtype: torch.dtype | None = None,
    ) -> None:
        '''
        d_model: Dimensionality of the Transformer block inputs.
            == embedding vector dimension size d_embedding.
        num_heads: Number of heads to use in multi-head self-attention.
        theta, max_seq_len: Parameters for initializing RoPE layer.

        Folllowing Vaswani et al. [2017], set d_k = d_v = d_model / h.

        TODO:
        - Any initialization for causal masking?
        
        '''
        super().__init__()
        # Initialize following linear weights per assignment section 3.4.1
        # W_Q, W_K, W_V all in shape (d_model (=h x d_k or h x d_v), d_model)
        # W_O in shape (d_model, d_model (=h x d_v))
        std = d_model ** -0.5
        self.W_Q = nn.Parameter(nn.init.trunc_normal_(torch.empty(d_model, d_model, device=device, dtype=dtype), mean=0, std=std, a=-3*std, b=3*std))
        self.W_K = nn.Parameter(nn.init.trunc_normal_(torch.empty(d_model, d_model, device=device, dtype=dtype), mean=0, std=std, a=-3*std, b=3*std))
        self.W_V = nn.Parameter(nn.init.trunc_normal_(torch.empty(d_model, d_model, device=device, dtype=dtype), mean=0, std=std, a=-3*std, b=3*std))
        self.W_O = nn.Parameter(nn.init.trunc_normal_(torch.empty(d_model, d_model, device=device, dtype=dtype), mean=0, std=std, a=-3*std, b=3*std))
        self.num_heads = num_heads
        if theta is not None and max_seq_len is not None:
            self.rope = RotaryPositionalEmbedding(theta, d_k=d_model, max_seq_len=max_seq_len, device=device, dtype=dtype)
        
    def forward(self, x: Float[Tensor, '... seq_len d_model']) -> torch.Tensor:
        '''
        Spec:
        (NOTE we use impl notation below instead of math notation appeared in assignment handout)
        
        Build the attention logic bottom-up, w/ naive approach first.
            The smallest logic unit is Attention(Q_i, K_i, V_i), where i in [0, num_heads)
            Q_i = x @ (the i-th out of h slices of W_Q.T). The shape of W_Q.T is (d_model, h x d_k)
            
            Similar paradigm applies to K_i and V_i. Reading the original transformer paper I believe
            the dimension on which slicing applies is the dimension of embedding vector.
            
            (Slicing this way makes sense, cuz suppose we sliced on the dimension of token sequence,
            then when processing a token in a particular slice, how can the model attend to tokens
            which run *before* this token? No way, cuz embeddings of such predecessor tokens likely
            would have been in different slices -- a contradiction and dead end)

        Dimension along which concatenation of head_1, head_2, ... head_h is the last dim
            and the size of resulting dimension is d_model

        IMO we only need to slice right before calling scaled_dot_product_attention fn.
        Ensure correctness first before jumping to optimization.

        TODO:
            - Masking to attend only predecessor tokens
        TODO: For efficient tensor ops, we can lump W_Q, W_K, W_V into a single large tensor instead of separating them.
        '''
        # Reshape tensors so that on the last dimension resides per-head values subject to application of attention.
        # This way we avoid the hassle and inefficiency of for i in num_heads do Attention(Q_i, K_i, V_i) :D
        # Note d_head x d_embedding_per_head = d_embedding (aka d_model)
        Q_all_h = rearrange(x @ self.W_Q.T, '... d_seq (d_head d_embedding_per_head) -> ... d_head d_seq d_embedding_per_head', d_head=self.num_heads)
        K_all_h = rearrange(x @ self.W_K.T, '... d_seq (d_head d_embedding_per_head) -> ... d_head d_seq d_embedding_per_head', d_head=self.num_heads)
        # Apply RoPE to query and key before applying attention
        if hasattr(self, 'rope'):
            Q_all_h = self.rope.forward(Q_all_h)
            K_all_h = self.rope.forward(K_all_h)
        V_all_h = rearrange(x @ self.W_V.T, '... d_seq (d_head d_embedding_per_head) -> ... d_head d_seq d_embedding_per_head', d_head=self.num_heads)
        # Tensor for causal masking
        # We shall directly use the existing masking logic in scaled_dot_product_attention; The problem is now reduced to creation of a mask tensor.
        # By definition, the mask should apply along the dimension of token sequence.
        # Prompting ChatGPT about the meaning of Q and K matrix, whose result of matmul over dimension (d_embedding_per_head)
        # (shape involved: (d_q x d_embedding_per_head) @ (d_embedding_per_head x d_k)) is
        # the input of masking in scaled dot product attention function, ChatGPT's response
        # claims that a row in Q, corresponding to a token in sequence, represents the "questions" / "queries"
        # which this particular token ask / attend to all other tokens in the sequence. HOWEVER,
        # I believe such interpretation should apply to the matmul result above instead; Let's say
        # the matmul result is QK (of shape (... d_q d_k)), focus on the 2-d matrix of shape (d_q d_k),
        # then the row at index i of this matrix, whose size is d_k, represents the "questions" / "queries"
        # of token i in the sequence to all other tokens in the same sequence. Item at index j of this
        # row represents the query of token i to token j in the same sequence. Meanwhile, column at index i
        # of this matrix represents the info which token at index i of sequence provides to all other tokens
        # in the same sequence. See https://chatgpt.com/s/t_6964464a2ff48191a4208206c18b4f06
        # In this light, we should create the mask so that the i-th token only
        # query all its predecessor, resulting in mask matrix whose elements of lower triangular part
        # are True while others are False
        # Note the dimension required here is the sequence length dimension which is not available in module init time :(
        d_seq_size = x.shape(-2)
        mask = torch.tril(torch.ones(d_seq_size, d_seq_size, device=device, dtype=torch.bool))
        attended_all_h: Float[Tensor, '... d_head d_seq d_embedding_per_head'] = scaled_dot_product_attention(Q_all_h, K_all_h, V_all_h, mask)
        # Concatenate to restore the dimension of embedding vector, so that it is ready for linear transformation w/ W_O
        attended_all_h = rearrange(attended_all_h, '... d_head d_seq d_embedding_per_head -> ... d_seq (d_head d_embedding_per_head)')
        # output of shape (... d_seq d_model)
        return attended_all_h @ self.W_O.T


In [239]:
# TODO Toying time: Pytorch idiomatic way to slice 2-D tensor into h pieces :D
t = torch.arange(0, 18).reshape(3, 6)
print(t)
indices = torch.arange(0, 6, 2, dtype=torch.int)
print(indices, indices+2)
t[..., indices:indices+2]

tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16, 17]])
tensor([0, 2, 4], dtype=torch.int32) tensor([2, 4, 6], dtype=torch.int32)


TypeError: only integer tensors of a single element can be converted to an index

In [248]:
from einops import rearrange
# NOTE correct way of slicing
rearrange(t, 'seq (head chunk_in_head) -> head seq chunk_in_head', head = 3)

tensor([[[ 0,  1],
         [ 6,  7],
         [12, 13]],

        [[ 2,  3],
         [ 8,  9],
         [14, 15]],

        [[ 4,  5],
         [10, 11],
         [16, 17]]])

In [263]:
mask = torch.tril(torch.ones(3, 3, dtype=torch.bool))
print(f'mask:\n{mask}')

# tensor whose values will be masked
target = torch.arange(0, 3.6, 0.2).reshape(2, 3, 3)
print(f'target before masking:\n{target}')

mask:
tensor([[ True, False, False],
        [ True,  True, False],
        [ True,  True,  True]])
target before masking:
tensor([[[0.0000, 0.2000, 0.4000],
         [0.6000, 0.8000, 1.0000],
         [1.2000, 1.4000, 1.6000]],

        [[1.8000, 2.0000, 2.2000],
         [2.4000, 2.6000, 2.8000],
         [3.0000, 3.2000, 3.4000]]])


In [264]:
# Correct; But we cannot use mask as a selector of target (aka target[mask] will fail)
target[..., ~mask] = -float('inf')
print(target)
target[mask]

tensor([[[0.0000,   -inf,   -inf],
         [0.6000, 0.8000,   -inf],
         [1.2000, 1.4000, 1.6000]],

        [[1.8000,   -inf,   -inf],
         [2.4000, 2.6000,   -inf],
         [3.0000, 3.2000, 3.4000]]])


IndexError: The shape of the mask [3, 3] at index 0 does not match the shape of the indexed tensor [2, 3, 3] at index 0

In [270]:
from einops import repeat

t = torch.arange(0, 6).reshape(2, 3)
print(f'repeat along row dim:\n', repeat(t, 'row col -> (rep row) col', rep = 2))
print(f'repeat along col dim:\n', repeat(t, 'row col -> row (rep col)', rep = 2))

repeat along row dim:
 tensor([[0, 1, 2],
        [3, 4, 5],
        [0, 1, 2],
        [3, 4, 5]])
repeat along col dim:
 tensor([[0, 1, 2, 0, 1, 2],
        [3, 4, 5, 3, 4, 5]])


## Gauge compute resource a Transformer model of given spec is to use

In [83]:
from math import prod
import torch

# Gauge result
Result = namedtuple("Result", ["flops", "mem"])


class Gauge:
    """
    Gauge the compute resource to be used by a Transformer model of given specs.
    Supported compute resource types:
    - FLOPs
    - Memory usage

    Assume the Transformer model being measured uses RoPE.

    Calculation of memory usage is more subtle: Naive summation of memory
    usaged in each compute component IMO doesn't make sense, as memory (either
    machine's physical RAM or accelerator's on-device mem) got allocated and
    released as computation comes and goes. IMO it makes more sense to measure
    the size of data which remains in memory during forward and backward pass
    of the model.
    """

    def __init__(
        self,
        input_shape: tuple[int, ...],
        vocab_size: int,
        context_len: int,
        num_layers: int,
        d_model: int,
        num_heads: int,
        d_ff: int,
        dtype: torch.dtype | None = torch.float32,
    ) -> None:
        """
        input_shape: Tuple representing the shape of input to Transformer
            model. Its length represents the number of input's dimensions,
            and its i-th item represents the size of input's i-th dimension
            (aka number of elements *on* i-th dimension, not including
            those in sub-dimensions). Typically (batch_size seq_len)
        """
        self.input_shape = input_shape
        self.vocab_size = vocab_size
        self.context_len = context_len
        self.num_layers = num_layers
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_ff = d_ff
        assert dtype is not None
        self.dtype = dtype

    def trainable_params(self) -> int:
        '''
        Return total # trainable params of the Transformer model.
    
        Spec:
            - Per Transformer block, we have 2 RMSNorms, 1 MHA and 1 FFN.
              # params = 2 * d_model + 4 * d_model ** 2 + 3 * (d_model * d_ff)
            - Besides Transformer blocks, we have 1 post transformer block
              RMSNorm layer, 1 linear layer for output embedding.
              # params = d_model + d_model * vocab_size
            So total trainable params = num_layers * (2 * d_model + 4 * d_model ** 2
                + 3 * (d_model * d_ff)) + d_model * (1 + vocab_size)
        '''
        return self.num_layers * (2 * self.d_model + 4 * self.d_model ** 2 + 3 *
                              (self.d_model * self.d_ff)) + self.d_model * (1 + self.vocab_size)

    def gauge(self) -> Result:
        """
        Spec:

        Assume:
            - Operations e.g. rearrange, indexing cost no FLOPs.

        Est. FLOPs required
            = num_layer x FLOPs per transformer block +
              1 RMSNorm over x's dims +
              1 linear transform on x +
              1 softmax on linear transformed result

        TODO Est. memory required - Need to reconsider whether it
        really makes sense to return mem usage in each subsequent
        gauge_ methods.
        """
        # Input embedding layer occupies memory space
        mem_input_embedding = self.vocab_size * self.d_model * self.dtype.itemsize
        # TODO need to account for the new tensor created from advanced indexing?

        transformer_block_input_shape = (*self.input_shape, self.d_model)
        # FLOPs and mem usage per transformer block
        flops_block, mem_block = self.gauge_transformer_block(
            transformer_block_input_shape,
            self.num_heads,
            self.d_ff,
            self.dtype,
        )
        flops_norm, mem_norm = self.gauge_rms_norm(
            transformer_block_input_shape,
        )
        flops_lnr, mem_lnr = self.gauge_linear(
            transformer_block_input_shape,
            self.vocab_size,
            self.dtype,
        )
        flops_sm, mem_sm = self.gauge_softmax(
            transformer_block_input_shape,
            dim=-1,
        )
        flops_total = self.num_layers * flops_block + flops_norm + flops_lnr + flops_sm
        mem_total = mem_input_embedding + self.num_layers * mem_block + mem_norm + mem_lnr + mem_sm
        return Result(flops=flops_total, mem=mem_total)

    def gauge_transformer_block(
        self,
        input_shape: tuple[int, ...],
        num_heads: int,
        d_ff: int,
        dtype: torch.dtype,
    ) -> Result:
        """
        Est. FLOPs required =
            2 RMSNorm on x's dim +
            1 multihead attention on x's dim +
            2 elementwise add on x's dim +
            1 FFN on x's dim
        """
        flops_norm, mem_norm = self.gauge_rms_norm(input_shape)
        flops_mha, mem_mha = self.gauge_multihead_attention(input_shape,
                                                            num_heads, dtype)
        flops_ffn, mem_ffn = self.gauge_ffn(input_shape, d_ff, dtype)
        flops_total = 2 * flops_norm + flops_mha + flops_ffn + 2 * prod(input_shape)
        mem_total = mem_mha + mem_ffn
        return Result(flops=flops_total, mem=mem_total)

    def gauge_rms_norm(self, input_shape: tuple[int, ...]) -> Result:
        """
        Est. FLOPs required =
            1 elementwise square on x +
            1 summation along x's last dim +
            1 division + 1 addition + 1 sqrt on reduced x, elementwise +
            1 mul + 1 div on x, elementwise
        """
        input_size = prod(input_shape)
        flops_sum = 0
        # elementwise square
        flops_sum += input_size
        reduced_input_size = prod(input_shape[:-1])
        # reduce by sum along input's last dim
        flops_sum += reduced_input_size * (input_shape[-1] - 1)
        # ops on reduced input
        flops_sum += 3 * reduced_input_size
        flops_sum += 2 * input_size
        # TODO memory usage?
        return Result(flops=flops_sum, mem=0)

    def gauge_linear(self, input_shape: tuple[int, ...], d_out: int, dtype:
                     torch.dtype) -> Result:
        """
        It is more intuitive to consider first how many items there are in the
        resultant matrix after linear transformation, then consider how many
        ops are spent to calculate each item.

        Est. FLOPs required = resultant matrix size x ([# mul ops] size of dim reduced + [# add ops] size of dim reduced - 1)
            = prod(input's dims except last one, d_out) x (2 x size of input's last dim - 1)
        """
        reduced_dim_size = input_shape[-1]
        result_size = prod(input_shape[:-1]) * d_out
        flops = result_size * (2 * reduced_dim_size - 1)

        # numel = number of tensor elements in memory
        # linear layer weight size
        numel = (reduced_dim_size * d_out)
        # resultant matrix size
        numel += result_size * dtype.itemsize
        return Result(flops=flops, mem=numel * dtype.itemsize)

    def gauge_softmax(self, input_shape: tuple[int, ...], dim: int) -> Result:
        """
        dim: Index of input dimension to apply softmax, size of this dimension
            is `input_shape[dim]`

        FLOPs estimate only applies to the naive softmax implemented here.

        Est. FLOPs required =
            1 max over x's i-th dim +
            1 elementwise sub and exp on x's dims +
            1 summation on x's i-th dim +
            1 elementwise div on x's dims
        """
        input_size = prod(input_shape)
        softmaxed_dim_size = input_shape[dim]
        # max and summation over dimension dim
        flops = 2 * input_size / softmaxed_dim_size * (softmaxed_dim_size - 1)
        flops += 3 * input_size
        # TODO memory usage?
        return Result(flops=flops, mem=0)

    def gauge_multihead_attention(
            self, input_shape: tuple[int, ...], num_heads: int, dtype:
            torch.dtype
    ) -> Result:
        """
        Assume:
            - d_k = k_v = d_model / num_heads
        Est. FLOPs required (assumed application of RoPE) =
            2 x prod(x's prefixing dims) x d_seq x 3 x d_model x d_model +
            3 x prod(rope function input dims, aka (... 2 d_head d_seq d_embedding_per_head) ) +
            scaled dot attention layer +
                2 * Q.numel() * seq_len * (2 + 3/d_k)
                = 2 * prod(... d_head d_seq d_embedding_per_head) * d_seq * (2 + 3 * h / d_model)
            tensor stacking allocates mem and returns a new tensor +
            matmul w/ W_O
                2 x prod(... d_seq d_model) x d_model
        """
        # batched matmul b/w x and QKV
        d_model, d_seq = input_shape[-1], input_shape[-2]
        input_size = prod(input_shape)
        flops, mem = (3 * v for v in self.gauge_linear(input_shape, d_model,
                                                       dtype))

        # RoPE on Q, K
        flops += 3 * (input_size * 2)
        # RoPE layer precomputed angle data
        # TODO currently we create RotaryPositionalEmbedding module for each
        # transformer block and repeatedly compute the sin and cos of the same
        # rotation angle. For efficient use of memory space, is there a way to
        # share such data across multiple nn modules?
        # Memory usage by RoPE module
        mem += (self.context_len * d_model) * dtype.itemsize
        # NOTE mask tensor data type = torch.bool whose element occupies 1 byte
        mem_attn_mask = d_seq**2
        mem += mem_attn_mask

        batched_xq_dims = (
            *input_shape[:-2],
            self.num_heads,
            d_seq,
            d_model // self.num_heads,
        )
        flops_attn, mem_attn = self.gauge_scaled_dot_product_attention(
            batched_xq_dims,
            batched_xq_dims,
            batched_xq_dims,
            dtype,
        )
        flops += flops_attn
        mem += mem_attn
        # pytorch tensor stacking allocates and returns a new tensor of the same shape as input
        mem += input_size * dtype.itemsize
        # matmul w/ W_O
        flops_outer_matmul, mem_outer_matmul = self.gauge_linear(input_shape,
                                                                 d_model, dtype)
        flops += flops_outer_matmul
        mem += mem_outer_matmul
        return Result(flops=flops, mem=mem)

    def gauge_scaled_dot_product_attention(
        self,
        Q_shape: tuple[int, ...],
        K_shape: tuple[int, ...],
        V_shape: tuple[int, ...],
        dtype: torch.dtype,
    ) -> Result:
        """
        Input of scaled_dot_product_attention function as follow:
            Q: Float[Tensor, "... queries d_k"],
            K: Float[Tensor, "... keys d_k"],
            V: Float[Tensor, "... values d_v"],
            mask: Bool[Tensor, "... queries keys"] | None = None,

        Here we assume queries = keys = d_seq

        Est. FLOPs required =
            1 matmul(Q, K) +
            1 elementwise div at dim of matmul(Q, K) +
            1 softmax at at dim of matmul(Q, K) +
            1 matmul(matmul(Q, K), V)
            In practice, d_k = d_v and keys = seq_len, so reduce above to
            following:
            = 2 * Q.numel() * seq_len * (2 + 3/d_k)

        """
        # batched matmul b/w Q, K
        flops, mem = self.gauge_linear(Q_shape, K_shape[-2], dtype)
        # NOTE gauge_linear accounts for size of linear transform weight when
        # compute memory usage. Here however we need to discount it, as the
        # weights have been accounted for in gauge_multihead_attention(),
        # precisely its very 1st call to gauge_linear(). Still we need to
        # account for the size of new matrix returned from linear transform.
        # The same applies to below.
        mem -= (Q_shape[-1] * K_shape[-2]) * dtype.itemsize
        QK_shape = (*Q_shape[:-1], K_shape[-2])
        flops_sm, mem_sm = self.gauge_softmax(QK_shape, dim=-1)
        flops += flops_sm
        # batched matmul b/w QK, V
        flops_qkv, mem_qkv = self.gauge_linear(QK_shape, V_shape[-1], dtype)
        flops += flops_qkv
        mem_qkv -= (QK_shape[-1] * V_shape[-1]) * dtype.itemsize
        mem += mem_qkv
        return Result(flops=flops, mem=mem)

    def gauge_ffn(self, input_shape: tuple[int, ...], d_ff: int, dtype:
                  torch.dtype) -> Result:
        """
        Est. FLOPs required =
            matmul(x, w1) +
            elementwise sigmoid + elementwise mul of matmul(x, w1) +
            matmul(x, w3) +
            elementwise mul of matmul(x, w1) +
            matmul w/ w2
        """
        d_model = input_shape[-1]
        # matmul(x, w1) and matmul(x, w3)
        flops, mem = (v * 2 for v in self.gauge_linear(input_shape, d_ff, dtype))
        inner_mat_shape = (*input_shape[:-1], d_ff)
        # elementwise sigmoid (4 ops per its math formula) and 2 mul of inner matrix
        flops += (4 + 2) * prod(inner_mat_shape)
        # matmul w/ w2
        flops_outer_matmul, mem_outer_matmul = self.gauge_linear(
            inner_mat_shape, d_model, dtype
        )
        flops += flops_outer_matmul
        mem += mem_outer_matmul
        return Result(flops, mem)


In [87]:
tensor_dtype = torch.float32

# GPT2 XL
g = Gauge(
    input_shape=(1, 1024), # batch=3 seq_len=16
    vocab_size=50257,
    context_len=1024,
    num_layers=48,
    d_model=1600,
    num_heads=25,
    d_ff=6400,
    dtype=tensor_dtype)

print(g.gauge())
print(g.gauge_transformer_block(input_shape=(64, 128, 128), num_heads=8, d_ff=64, dtype=tensor_dtype))
print(g.trainable_params())

Result(flops=4520375434240.0, mem=47759233536)
Result(flops=2117771264.0, mem=541573120)
2046646400


In [ ]:
# BERT
g = Gauge(
    input_shape=(1, 1024), # batch=3 seq_len=16
    vocab_size=30000,
    context_len=1024,
    num_layers=48,
    d_model=1600,
    num_heads=25,
    d_ff=6400,
    dtype=tensor_dtype)

2046646400

In [71]:
import torch

torch.manual_seed(14)

# toy input embedding tensor
t1 = torch.arange(0, 24).reshape(8, 3)
# batched token indices batch = 2 seq_len = 4
indices = torch.randint(0, 8, (2, 4))

print(t1)
print(indices)

selected = t1[indices]
print(selected, selected.shape)

selected[0, 0, 0] = -1024
# this confirms selected is a newly created tensor, not a view of t1
print(selected)
print(t1)

tensor([[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11],
        [12, 13, 14],
        [15, 16, 17],
        [18, 19, 20],
        [21, 22, 23]])
tensor([[3, 0, 4, 6],
        [7, 1, 6, 2]])
tensor([[[ 9, 10, 11],
         [ 0,  1,  2],
         [12, 13, 14],
         [18, 19, 20]],

        [[21, 22, 23],
         [ 3,  4,  5],
         [18, 19, 20],
         [ 6,  7,  8]]]) torch.Size([2, 4, 3])
tensor([[[-1024,    10,    11],
         [    0,     1,     2],
         [   12,    13,    14],
         [   18,    19,    20]],

        [[   21,    22,    23],
         [    3,     4,     5],
         [   18,    19,    20],
         [    6,     7,     8]]])
tensor([[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11],
        [12, 13, 14],
        [15, 16, 17],
        [18, 19, 20],
        [21, 22, 23]])


## Numpy Indexing

In [218]:
import numpy as np

arr = np.arange(10)
print(arr)
arr.shape = (2, 5)
print(arr)

print(arr[0, 4], arr[0][4])

arr.shape = (10,)
arr[1:7:2]

[0 1 2 3 4 5 6 7 8 9]
[[0 1 2 3 4]
 [5 6 7 8 9]]
4 4


array([1, 3, 5])

In [220]:
arr[::], arr[:]

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))

In [226]:
x = np.array([[[1, 0.1],[2, 0.2],[3, 0.3]], [[4, 0.4],[5, 0.5],[6, 0.6]]])
print(x.shape, x.ndim)
# https://numpy.org/doc/stable/user/basics.indexing.html#dimensional-indexing-tools
print(x[..., 0], x[..., 0].shape)
print(x[..., 1], x[..., 1].shape)

(2, 3, 2) 3
[[1. 2. 3.]
 [4. 5. 6.]] (2, 3)
[[0.1 0.2 0.3]
 [0.4 0.5 0.6]] (2, 3)


> (Numpy) Advanced indexing always returns a copy of the data (contrast with basic slicing that returns a view).

## Play matrix multiply

In [21]:
import torch
from einops import einsum

# (batch_size seq_len d_model)
x = torch.arange(0, 12, 1).to(torch.float32).reshape(2, 2, 3)
print(x)
# spec: Use 2 tensors, one for Q,K,V and one for W_O
# qkv of shape (3 d_model d_model)
# q, k, v = qkv[0], qkv[1], qkv[2]
qkv = torch.arange(0, 2.7, 0.1).reshape(3, 3, 3)
print(qkv)
print(f'q = {qkv[0]}, q shape = {qkv[0].shape}')

# mimic x @ W_Q.T, x @ W_K.T and x @ W_V.T in one go
# Expect: matmul in one go: (... seq_len d_model) x (3 d_model d_model) -> (... 3 seq_len d_model)
XQKV = einsum(x, qkv, '... seq d_model, kind d_to_split_by_head d_model -> ... kind seq d_to_split_by_head')
XQKV, XQKV.shape

tensor([[[ 0.,  1.,  2.],
         [ 3.,  4.,  5.]],

        [[ 6.,  7.,  8.],
         [ 9., 10., 11.]]])
tensor([[[0.0000, 0.1000, 0.2000],
         [0.3000, 0.4000, 0.5000],
         [0.6000, 0.7000, 0.8000]],

        [[0.9000, 1.0000, 1.1000],
         [1.2000, 1.3000, 1.4000],
         [1.5000, 1.6000, 1.7000]],

        [[1.8000, 1.9000, 2.0000],
         [2.1000, 2.2000, 2.3000],
         [2.4000, 2.5000, 2.6000]]])
q = tensor([[0.0000, 0.1000, 0.2000],
        [0.3000, 0.4000, 0.5000],
        [0.6000, 0.7000, 0.8000]]), q shape = torch.Size([3, 3])


(tensor([[[[ 0.5000,  1.4000,  2.3000],
           [ 1.4000,  5.0000,  8.6000]],
 
          [[ 3.2000,  4.1000,  5.0000],
           [12.2000, 15.8000, 19.4000]],
 
          [[ 5.9000,  6.8000,  7.7000],
           [23.0000, 26.6000, 30.2000]]],
 
 
         [[[ 2.3000,  8.6000, 14.9000],
           [ 3.2000, 12.2000, 21.2000]],
 
          [[21.2000, 27.5000, 33.8000],
           [30.2000, 39.2000, 48.2000]],
 
          [[40.1000, 46.4000, 52.7000],
           [57.2000, 66.2000, 75.2000]]]]),
 torch.Size([2, 3, 2, 3]))

In [27]:
# get XQ and XK
XQ = XQKV[:, 0]
print(XQ, XQ.shape)
XK = XQKV[:, 1]
print(XK, XK.shape)
XQ_and_XK = XQKV[:, 0:2]
print(XQ_and_XK, XQ_and_XK.shape)

tensor([[[ 0.5000,  1.4000,  2.3000],
         [ 1.4000,  5.0000,  8.6000]],

        [[ 2.3000,  8.6000, 14.9000],
         [ 3.2000, 12.2000, 21.2000]]]) torch.Size([2, 2, 3])
tensor([[[ 3.2000,  4.1000,  5.0000],
         [12.2000, 15.8000, 19.4000]],

        [[21.2000, 27.5000, 33.8000],
         [30.2000, 39.2000, 48.2000]]]) torch.Size([2, 2, 3])
tensor([[[[ 0.5000,  1.4000,  2.3000],
          [ 1.4000,  5.0000,  8.6000]],

         [[ 3.2000,  4.1000,  5.0000],
          [12.2000, 15.8000, 19.4000]]],


        [[[ 2.3000,  8.6000, 14.9000],
          [ 3.2000, 12.2000, 21.2000]],

         [[21.2000, 27.5000, 33.8000],
          [30.2000, 39.2000, 48.2000]]]]) torch.Size([2, 2, 2, 3])


In [34]:
# equivalent to XQ_and_XK, by slicing and selecting along the 3rd-from-last dimension
XQKV[..., 0:2, :, :]

tensor([[[[ 0.5000,  1.4000,  2.3000],
          [ 1.4000,  5.0000,  8.6000]],

         [[ 3.2000,  4.1000,  5.0000],
          [12.2000, 15.8000, 19.4000]]],


        [[[ 2.3000,  8.6000, 14.9000],
          [ 3.2000, 12.2000, 21.2000]],

         [[21.2000, 27.5000, 33.8000],
          [30.2000, 39.2000, 48.2000]]]])

In [28]:
help(torch.stack)

Help on built-in function stack in module torch:

stack(...)
    stack(tensors, dim=0, *, out=None) -> Tensor

    Concatenates a sequence of tensors along a new dimension.

    All tensors need to be of the same size.

    .. seealso::

        :func:`torch.cat` concatenates the given sequence along an existing dimension.

    Arguments:
        tensors (sequence of Tensors): sequence of tensors to concatenate
        dim (int): dimension to insert. Has to be between 0 and the number
            of dimensions of concatenated tensors (inclusive)

    Keyword args:
        out (Tensor, optional): the output tensor.



In [ ]:
# tests/test_model.py::test_multihead_self_attention_with_rope x_even_idx shape: torch.Size([4, 4, 12, 8]) cos_by_pos shape: torch.Size([1, 1, 12, 32])

In [9]:
t = torch.arange(0, 24).reshape(6, 4)
print(t, t.shape)
selector = torch.tensor([[0, 1, 2, 3, 4, 5], [2, 1, 3, 0, 4, 5]])
print(f'selector tensor shape: {selector.shape}')
ts = t[selector]
print(ts, ts.shape)

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15],
        [16, 17, 18, 19],
        [20, 21, 22, 23]]) torch.Size([6, 4])
selector tensor shape: torch.Size([2, 6])
tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11],
         [12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]],

        [[ 8,  9, 10, 11],
         [ 4,  5,  6,  7],
         [12, 13, 14, 15],
         [ 0,  1,  2,  3],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]]) torch.Size([2, 6, 4])


## Test nn.Module w/ multi-return values from its `forward()` method

In [8]:
import torch
from torch import nn

class Toy1(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        print(f'Toy1: x = {x}')
        return x, torch.tensor([1, 2, 3])

class Toy2(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x, pos = None):
        print(f'Toy2: x = {x} pos = {pos}')
        return torch.zeros(2, 2)


toy_seq = nn.Sequential(Toy1(), Toy2())
toy_seq(torch.tensor([9, 8, 7]))

Toy1: x = tensor([9, 8, 7])
Toy2: x = (tensor([9, 8, 7]), tensor([1, 2, 3])) pos = None


tensor([[0., 0.],
        [0., 0.]])

In [10]:
toy_module_list = nn.ModuleList([Toy1(), Toy2()])
toy_module_list(torch.tensor([9, 8]))

NotImplementedError: Module [ModuleList] is missing the required "forward" function

In [44]:
from collections import namedtuple

t1 = (1, 2)
t2 = (*t1, 3)
print(t2)
print(t2[:-1])
print(2 * t2)

(1, 2, 3)
(1, 2)
(1, 2, 3, 1, 2, 3)


In [43]:
import torch

t1 = torch.arange(0, 2.4, 0.1).reshape(4, 2, 3)
t2 = torch.arange(1, 7, 1).to(torch.float32).reshape(3, 2)
print(t1)
print(t2)
print(t1 @ t2)

tensor([[[0.0000, 0.1000, 0.2000],
         [0.3000, 0.4000, 0.5000]],

        [[0.6000, 0.7000, 0.8000],
         [0.9000, 1.0000, 1.1000]],

        [[1.2000, 1.3000, 1.4000],
         [1.5000, 1.6000, 1.7000]],

        [[1.8000, 1.9000, 2.0000],
         [2.1000, 2.2000, 2.3000]]])
tensor([[1., 2.],
        [3., 4.],
        [5., 6.]])
tensor([[[ 1.3000,  1.6000],
         [ 4.0000,  5.2000]],

        [[ 6.7000,  8.8000],
         [ 9.4000, 12.4000]],

        [[12.1000, 16.0000],
         [14.8000, 19.6000]],

        [[17.5000, 23.2000],
         [20.2000, 26.8000]]])


In [45]:
torch.bool.itemsize

1

## Pytorch tensor stacking allocates mem and returns a new tensor


In [49]:
import torch

t1 = torch.arange(0, 6).reshape(2, 3)
t2 = torch.arange(6, 12).reshape(2, 3)

t3 = torch.stack([t1, t2])
print(t3)

t1[0, 0] = -100
print(t1)
print(t3)

tensor([[[ 0,  1,  2],
         [ 3,  4,  5]],

        [[ 6,  7,  8],
         [ 9, 10, 11]]])
tensor([[-100,    1,    2],
        [   3,    4,    5]])
tensor([[[ 0,  1,  2],
         [ 3,  4,  5]],

        [[ 6,  7,  8],
         [ 9, 10, 11]]])


# Scratch: Pytorch tutorial

In [46]:
# -*- coding: utf-8 -*-
import numpy as np
import math

# Create random input and output data
x = np.linspace(-math.pi, math.pi, 2000)
y = np.sin(x)

# Randomly initialize weights
a = np.random.randn()
b = np.random.randn()
c = np.random.randn()
d = np.random.randn()

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y
    # y = a + b x + c x^2 + d x^3
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss
    loss = np.square(y_pred - y).sum()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of a, b, c, d with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # Update weights
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d

print(f'Result: y = {a} + {b} x + {c} x^2 + {d} x^3')

99 936.250643272073
199 645.6479100440048
299 446.59769261270117
399 310.1005226218571
499 216.39180309800014
599 151.98552907638506
699 107.66922492043156
799 77.14248585274315
899 56.091513441801744
999 41.55931924194568
1099 31.516652391540806
1199 24.569345205417985
1299 19.75847589733717
1399 16.423754124106054
1499 14.110020007413638
1599 12.50317712048318
1699 11.386243389773334
1799 10.609168273305954
1899 10.068080749363528
1999 9.691005440031622
Result: y = -0.028238700554603 + 0.8443640706625123 x + 0.004871645739674506 x^2 + -0.09156990094297779 x^3


In [50]:
import torch
import math

# We want to be able to train our model on an `accelerator <https://pytorch.org/docs/stable/torch.html#accelerators>`__
# such as CUDA, MPS, MTIA, or XPU. If the current accelerator is available, we will use it. Otherwise, we use the CPU.

dtype = torch.float
#device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
#print(f"Using {device} device")
# Surprisingly my old MBP still has MPS support:
# https://developer.apple.com/metal/pytorch/
torch.set_default_device('mps')

# Create Tensors to hold input and outputs.
# By default, requires_grad=False, which indicates that we do not need to
# compute gradients with respect to these Tensors during the backward pass.
x = torch.linspace(-1, 1, 2000, dtype=dtype)
y = torch.exp(x) # A Taylor expansion would be 1 + x + (1/2) x**2 + (1/3!) x**3 + ...

# Create random Tensors for weights. For a third order polynomial, we need
# 4 weights: y = a + b x + c x^2 + d x^3
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
a = torch.randn((), dtype=dtype, requires_grad=True)
b = torch.randn((), dtype=dtype, requires_grad=True)
c = torch.randn((), dtype=dtype, requires_grad=True)
d = torch.randn((), dtype=dtype, requires_grad=True)

initial_loss = 1.
learning_rate = 1e-5
for t in range(5000):
    # Forward pass: compute predicted y using operations on Tensors.
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()

    # Calculare initial loss, so we can report loss relative to it
    if t==0:
        initial_loss=loss.item()

    if t % 100 == 99:
        print(f'Iteration t = {t:4d}  loss(t)/loss(0) = {round(loss.item()/initial_loss, 6):10.6f}  a = {a.item():10.6f}  b = {b.item():10.6f}  c = {c.item():10.6f}  d = {d.item():10.6f}')

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call a.grad, b.grad. c.grad and d.grad will be Tensors holding
    # the gradient of the loss with respect to a, b, c, d respectively.
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # Manually zero the gradients after updating weights
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

Iteration t =   99  loss(t)/loss(0) =   0.358655  a =   0.675835  b =   0.660643  c =   1.421510  d =   0.588552
Iteration t =  199  loss(t)/loss(0) =   0.192731  a =   0.763050  b =   0.725041  c =   1.180236  d =   0.589011
Iteration t =  299  loss(t)/loss(0) =   0.108448  a =   0.826638  b =   0.749254  c =   1.004798  d =   0.566660
Iteration t =  399  loss(t)/loss(0) =   0.063148  a =   0.872891  b =   0.766290  c =   0.877192  d =   0.542195
Iteration t =  499  loss(t)/loss(0) =   0.038468  a =   0.906533  b =   0.781354  c =   0.784376  d =   0.518750
Iteration t =  599  loss(t)/loss(0) =   0.024785  a =   0.931004  b =   0.795313  c =   0.716866  d =   0.496726
Iteration t =  699  loss(t)/loss(0) =   0.016996  a =   0.948802  b =   0.808353  c =   0.667761  d =   0.476106
Iteration t =  799  loss(t)/loss(0) =   0.012395  a =   0.961748  b =   0.820550  c =   0.632045  d =   0.456811
Iteration t =  899  loss(t)/loss(0) =   0.009540  a =   0.971165  b =   0.831962  c =   0.606066

In [54]:
x = torch.linspace(-math.pi, math.pi, 2000)
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)

In [55]:
print(x.shape)
x.unsqueeze(-1).shape

torch.Size([2000])


torch.Size([2000, 1])

In [51]:
import numpy as np

x = np.random.RandomState(42).normal(size=[10, 32, 100, 200])

In [61]:
x_listed = list(x)
rearrange(x_listed, 'b c h w -> b c h w').shape

(10, 32, 100, 200)

In [64]:
from einops import rearrange

tensors = list([
    torch.tensor([[1, 2], [3,4]]),
    torch.tensor([[5, 6], [7, 8]]),
])
rearrange(tensors, 'i j k -> i j k').shape

torch.Size([2, 2, 2])

In [73]:
# batch size, seq_len, embedding
torch.manual_seed(42)
token_embeddings = torch.randn(3, 4, 2)
token_embeddings

tensor([[[ 1.9269,  1.4873],
         [ 0.9007, -2.1055],
         [ 0.6784, -1.2345],
         [-0.0431, -1.6047]],

        [[ 0.3559, -0.6866],
         [-0.4934,  0.2415],
         [-1.1109,  0.0915],
         [-2.3169, -0.2168]],

        [[-0.3097, -0.3957],
         [ 0.8034, -0.6216],
         [-0.5920, -0.0631],
         [-0.8286,  0.3309]]])

In [97]:
# pos, d_embedding, d_embedding
dummy_rotation_mat = rearrange([torch.eye(2) * i * 0.1 for i in range(4)], '... -> ...')
dummy_rotation_mat

tensor([[[0.0000, 0.0000],
         [0.0000, 0.0000]],

        [[0.1000, 0.0000],
         [0.0000, 0.1000]],

        [[0.2000, 0.0000],
         [0.0000, 0.2000]],

        [[0.3000, 0.0000],
         [0.0000, 0.3000]]])

In [98]:
# batch, seq_len
torch.manual_seed(42)
token_positions = torch.randint(low=0, high=4, size=(3, 4))
token_positions

tensor([[2, 3, 0, 2],
        [2, 3, 0, 0],
        [2, 1, 2, 2]])

In [100]:
selected_rotation_mats = dummy_rotation_mat[token_positions]
selected_rotation_mats

tensor([[[[0.2000, 0.0000],
          [0.0000, 0.2000]],

         [[0.3000, 0.0000],
          [0.0000, 0.3000]],

         [[0.0000, 0.0000],
          [0.0000, 0.0000]],

         [[0.2000, 0.0000],
          [0.0000, 0.2000]]],


        [[[0.2000, 0.0000],
          [0.0000, 0.2000]],

         [[0.3000, 0.0000],
          [0.0000, 0.3000]],

         [[0.0000, 0.0000],
          [0.0000, 0.0000]],

         [[0.0000, 0.0000],
          [0.0000, 0.0000]]],


        [[[0.2000, 0.0000],
          [0.0000, 0.2000]],

         [[0.1000, 0.0000],
          [0.0000, 0.1000]],

         [[0.2000, 0.0000],
          [0.0000, 0.2000]],

         [[0.2000, 0.0000],
          [0.0000, 0.2000]]]])

In [110]:
from einops import einsum

einsum(token_embeddings, selected_rotation_mats, '... d_k, ... d_k d_k -> ... d_k')

tensor([[[ 0.3854,  0.2975],
         [ 0.2702, -0.6317],
         [ 0.0000, -0.0000],
         [-0.0086, -0.3209]],

        [[ 0.0712, -0.1373],
         [-0.1480,  0.0724],
         [-0.0000,  0.0000],
         [-0.0000, -0.0000]],

        [[-0.0619, -0.0791],
         [ 0.0803, -0.0622],
         [-0.1184, -0.0126],
         [-0.1657,  0.0662]]])

In [111]:
help(torch.arange)

Help on built-in function arange in module torch:

arange(...)
    arange(start=0, end, step=1, *, out=None, dtype=None, layout=torch.strided, device=None, requires_grad=False) -> Tensor

    Returns a 1-D tensor of size :math:`\left\lceil \frac{\text{end} - \text{start}}{\text{step}} \right\rceil`
    with values from the interval ``[start, end)`` taken with common difference
    :attr:`step` beginning from `start`.

    Note that non-integer :attr:`step` is subject to floating point rounding errors when
    comparing against :attr:`end`; to avoid inconsistency, we advise subtracting a small epsilon from :attr:`end`
    in such cases.

    .. math::
        \text{out}_{{i+1}} = \text{out}_{i} + \text{step}

    Args:
        start (Number): the starting value for the set of points. Default: ``0``.
        end (Number): the ending value for the set of points
        step (Number): the gap between each pair of adjacent points. Default: ``1``.

    Keyword args:
        out (Tensor, opti

In [112]:
torch.arange(0, 0.5, 0.01)

tensor([0.0000, 0.0100, 0.0200, 0.0300, 0.0400, 0.0500, 0.0600, 0.0700, 0.0800,
        0.0900, 0.1000, 0.1100, 0.1200, 0.1300, 0.1400, 0.1500, 0.1600, 0.1700,
        0.1800, 0.1900, 0.2000, 0.2100, 0.2200, 0.2300, 0.2400, 0.2500, 0.2600,
        0.2700, 0.2800, 0.2900, 0.3000, 0.3100, 0.3200, 0.3300, 0.3400, 0.3500,
        0.3600, 0.3700, 0.3800, 0.3900, 0.4000, 0.4100, 0.4200, 0.4300, 0.4400,
        0.4500, 0.4600, 0.4700, 0.4800, 0.4900])

In [114]:
max_seq_len = 4
positions = torch.arange(max_seq_len)
print(f'positions = {positions}')

tensor([0, 1, 2, 3])

torch.Size([4, 1])

In [134]:
theta = torch.pi / 6

d_k = 6
# torch.arange(0, d_k, 2) enumerates all values of 2k-2 for k in [1, d_k/2]
exponents = torch.arange(0, d_k, 2) / d_k
# 1 / (theta^((2k-2)/d))
inv_exps = theta ** -exponents
print(f'inv_exps = {inv_exps}')

# shape (max_seq_len, 1) x shape (1, d_k/2). Here is (4, 1) x (1, 3) works by broadcasting
angles = positions[:, None] * inv_exps[None, :]
angles

inv_exps = tensor([1.0000, 1.2407, 1.5393])


tensor([[0.0000, 0.0000, 0.0000],
        [1.0000, 1.2407, 1.5393],
        [2.0000, 2.4814, 3.0787],
        [3.0000, 3.7221, 4.6180]])

In [137]:
sin, cos = torch.sin(angles), torch.cos(angles)

tensor([[ 1.0000,  1.0000,  1.0000],
        [ 0.5403,  0.3241,  0.0315],
        [-0.4161, -0.7899, -0.9980],
        [-0.9900, -0.8362, -0.0942]])

In [139]:
R = torch.zeros(max_seq_len, d_k, d_k)
# Fill 2×2 blocks
idx = torch.arange(0, d_k, 2)
R[:, idx, idx] = cos
R[:, idx, idx + 1] = -sin
R[:, idx + 1, idx] = sin
R[:, idx + 1, idx + 1] = cos

In [140]:
R

tensor([[[ 1.0000, -0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  1.0000, -0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  1.0000, -0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000]],

        [[ 0.5403, -0.8415,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.8415,  0.5403,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.3241, -0.9460,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.9460,  0.3241,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0315, -0.9995],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.9995,  0.0315]],

        [[-0.4161, -0.9093,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.9093, -0.4161,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000, -0.7899, -0.6133,  0.0000,  0.0000],
         [ 0.0000,  0